# Imports and General Settings

In [5]:
import os

import tensorflow as tf
import tensorflow.contrib as tf_contrib

import pathlib

import numpy as np

import random
from cleverhans.loss import SNNLCrossEntropy
from tensorflow.examples.tutorials.mnist import input_data

# which GPU to be used (0 is RTX, 1 or 2 are either of the Titan Xps)
gpu = "/GPU:0"

AUTOTUNE = tf.contrib.data.AUTOTUNE

tf.enable_eager_execution()

session_config = tf.ConfigProto()
session_config.gpu_options.allow_growth = True
session_config.allow_soft_placement = True

# session_config.log_device_placement = True
tf.keras.backend.set_session(tf.Session(config=session_config))

print(tf.test.gpu_device_name())



/device:GPU:0


# ResNet Utility Functions

In [6]:

# Xavier : tf_contrib.layers.xavier_initializer()
# He : tf_contrib.layers.variance_scaling_initializer()
# Normal : tf.random_normal_initializer(mean=0.0, stddev=0.02)
# l2_decay : tf_contrib.layers.l2_regularizer(0.0001)

weight_init = tf_contrib.layers.variance_scaling_initializer()
weight_regularizer = tf_contrib.layers.l2_regularizer(0.0001)


##################################################################################
# Layer
##################################################################################

def conv(x, channels, kernel=4, stride=2, padding='SAME', use_bias=True, scope='conv_0'):
    with tf.variable_scope(scope):
        x = tf.layers.conv2d(inputs=x, filters=channels,
                             kernel_size=kernel, kernel_initializer=weight_init,
                             kernel_regularizer=weight_regularizer,
                             strides=stride, use_bias=use_bias, padding=padding)

        return x

def fully_connected(x, units, use_bias=True, scope='fully_0'):
    with tf.variable_scope(scope):
        x = flatten(x)
        x = tf.layers.dense(x, units=units, kernel_initializer=weight_init, kernel_regularizer=weight_regularizer, use_bias=use_bias)

        return x

def resblock(x_init, channels, kernel=3, is_training=True, use_bias=True, downsample=False, scope='resblock') :
    with tf.variable_scope(scope) :

        x = batch_norm(x_init, is_training, scope='batch_norm_0')
        x = relu(x)

        if downsample :
            x = conv(x, channels, kernel=kernel, stride=2, use_bias=use_bias, scope='conv_0')
            x_init = conv(x_init, channels, kernel=1, stride=2, use_bias=use_bias, scope='conv_init')

        else :
            x = conv(x, channels, kernel=kernel, stride=1, use_bias=use_bias, scope='conv_0')

        x = batch_norm(x, is_training, scope='batch_norm_1')
        x = relu(x)
        x = conv(x, channels, kernel=kernel, stride=1, use_bias=use_bias, scope='conv_1')

        return x + x_init

def bottle_resblock(x_init, channels, is_training=True, use_bias=True, downsample=False, scope='bottle_resblock') :
    with tf.variable_scope(scope) :
        x = batch_norm(x_init, is_training, scope='batch_norm_1x1_front')
        shortcut = relu(x)

        x = conv(shortcut, channels, kernel=1, stride=1, use_bias=use_bias, scope='conv_1x1_front')
        x = batch_norm(x, is_training, scope='batch_norm_3x3')
        x = relu(x)

        if downsample :
            x = conv(x, channels, kernel=3, stride=2, use_bias=use_bias, scope='conv_0')
            shortcut = conv(shortcut, channels*4, kernel=1, stride=2, use_bias=use_bias, scope='conv_init')

        else :
            x = conv(x, channels, kernel=3, stride=1, use_bias=use_bias, scope='conv_0')
            shortcut = conv(shortcut, channels * 4, kernel=1, stride=1, use_bias=use_bias, scope='conv_init')

        x = batch_norm(x, is_training, scope='batch_norm_1x1_back')
        x = relu(x)
        x = conv(x, channels*4, kernel=1, stride=1, use_bias=use_bias, scope='conv_1x1_back')

        return x + shortcut


##################################################################################
# Sampling
##################################################################################

def flatten(x) :
    return tf.layers.flatten(x)

def global_avg_pooling(x):
    gap = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
    return gap

def avg_pooling(x) :
    return tf.layers.average_pooling2d(x, pool_size=2, strides=2, padding='SAME')

def max_pooling(x) :
    return tf.layers.max_pooling2d(x, pool_size=2, strides=2, padding='SAME')


##################################################################################
# Activation function
##################################################################################

def relu(x):
    return tf.nn.relu(x)


##################################################################################
# Normalization function
##################################################################################

def batch_norm(x, is_training=True, scope='batch_norm'):
    return tf_contrib.layers.batch_norm(x,
                                        decay=0.9, epsilon=1e-05,
                                        center=True, scale=True, updates_collections=None,
                                        is_training=is_training, scope=scope)

##################################################################################
# Loss function
##################################################################################

def classification_loss(logit, label) :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=logit))
    prediction = tf.equal(tf.argmax(logit, -1), tf.argmax(label, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

    return loss, accuracy


# ResNet-10

## Architecture

In [7]:
num_classes = 10

# ResNet-10 architecture
def resnet10_network(x, labels, conv_kernel=7, resblock_kernel=3, num_channels=4, n_classes=num_classes, reuse = False, is_training = True):
    with tf.variable_scope('Resnet-10', reuse = reuse), tf.device(gpu):
    
        
        print(x)
        
        x = tf.reshape(x, [-1,28,28,1])
        
        # channels
        x = conv(x, num_channels, kernel=conv_kernel, stride=1, padding='SAME')

        
        
        x = resblock(x, channels=num_channels, kernel=resblock_kernel, is_training=is_training, 
                      downsample=False, scope='resblock0_id1')
            
        x = resblock(x, channels=num_channels * 2, kernel=resblock_kernel, is_training=is_training, downsample=True, scope='resblock_conv1')

        x = resblock(x, channels=num_channels * 2, kernel=resblock_kernel, is_training=is_training, 
                      downsample=False, scope='resblock1_id1')
        
        x = resblock(x, channels=num_channels * 4, kernel=resblock_kernel, is_training=is_training, downsample=True, scope='resblock_conv2')

        x = resblock(x, channels=num_channels * 4, kernel=resblock_kernel, is_training=is_training, 
                      downsample=False, scope='resblock2_id1')
        
        x = batch_norm(x, is_training, scope='batch_norm')
        x = relu(x)
        
        print(x)

        x = fully_connected(x, units=n_classes, scope='logit')
        
        return x
    
def resnet10_model_fn(features, labels, mode, conv_kernel=7, resblock_kernel=3, num_channels=4):
    with tf.device(gpu):
        
        logits_train = resnet10_network(features, conv_kernel=conv_kernel, resblock_kernel=resblock_kernel, num_channels=num_channels, n_classes=num_classes, reuse=False, is_training=True)
        logits_test = resnet10_network(features, conv_kernel=conv_kernel, resblock_kernel=resblock_kernel, num_channels=num_channels, n_classes=num_classes, reuse=True, is_training=False)
        # find highest val (prediction)
        pred_classes = tf.arg_max(logits_test, dimension=1)

        # if in prediction mode
        if mode == tf.estimator.ModeKeys.PREDICT:
            
            return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
        
        
        # Define loss and optimizer
        
        # loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))

        
        loss_op = loss_op + SNNLCrossEntropy.optimized_temp_SNNL(logits_train, labels, 50.0, True)
    
        
        tf.summary.scalar("loss", loss_op)
        
        print(loss_op)
        
        # Evaluate model accuracy
        acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
        tf.summary.scalar("training_accuracy", acc_op[1])
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
            
            # the different ops for training, evaluating, ...
            estim_specs = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=pred_classes,
                loss=loss_op,
                train_op=train_op,
                eval_metric_ops={'accuracy': acc_op},
            ) 

            return estim_specs
        
        elif mode == tf.estimator.ModeKeys.EVAL:
   
            # the different ops for training, evaluating, ...
            estim_specs = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=pred_classes,
                loss=loss_op,
                eval_metric_ops={'accuracy': acc_op},
            ) 

            return estim_specs
       
        else:
            print("Something went wrong.")


## Experimentation

In [8]:

tf.logging.set_verbosity(tf.logging.INFO)

b_size = 64

learning_rate = 0.001

random_seed = 777

num_steps = 4000

tf.summary.FileWriterCache.clear()
        
config = tf.estimator.RunConfig(
    log_step_count_steps= 20,
    save_summary_steps= 50,
    save_checkpoints_secs= 5,
    model_dir="/tmp/resnet_mnist_snnl/with_snnl_loss/",
    tf_random_seed=random_seed,
    session_config=session_config)

model_fn = lambda features, labels, mode: resnet10_model_fn(features, labels, mode, conv_kernel=7, num_channels=8)

mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

# Define the input function for training
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=mnist.train.images, y=mnist.train.labels, batch_size=b_size, num_epochs=None, shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=mnist.test.images, y=mnist.test.labels, batch_size=b_size, num_epochs=None, shuffle=True)



model = tf.estimator.Estimator(model_fn=model_fn, config=config)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_steps)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, throttle_secs=5)

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
        
        

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/resnet_mnist_snnl/with_snnl_loss/', '_tf_random_seed': 777, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 5, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_co

(None, None)